In [1]:
import pandas as pd
import numpy as np
import chiTariffs
import heapq

In [48]:
df2= pd.read_csv('/Users/dguiraldes/Documents/ISCI/intro_smart_meters/SML-dataset/halfhourly_dataset/halfhourly_dataset/block_0.csv')
data=df2[(df2['LCLid']=='MAC000002') & (df2['tstp']>='2013-01-01') & (df2['tstp']<'2014-01-01')].reset_index()[['tstp','energy(kWh/hh)']]

In [49]:
data['energy(kWh/hh)']=data['energy(kWh/hh)'].astype('float')

In [50]:
data['tstp']=pd.to_datetime(data['tstp'])

In [5]:
params_BT1={
    'cfm': np.ones(12)*684.730,
    'csp': np.ones(12)*0.508,
    'tr':np.ones(12)*12.784,
    'en':np.ones(12)*78.911,
    'pw':np.ones(12)*15.676,
    'bpw':np.ones(12)*18.326, 
    'pw_win':np.ones(12)*31.353,
    'bpw_win':np.ones(12)*36.583
}
params_BT4_3={
    'cfm': np.ones(12)*796.080,
    'csp': np.ones(12)*0.508,
    'tr':np.ones(12)*12.784,
    'en':np.ones(12)*78.911,
    'ppw':np.ones(12)*3487.9,
    'ppw_ph':np.ones(12)*10187.410
}

In [102]:
data2=data.copy()
data2['month']=data2['tstp'].dt.date.apply(lambda dt: dt.replace(day=1))
data2['power']=data2['energy(kWh/hh)']/2
data2.columns=['tstp','energy','month','power']
data2['mm']=data2['tstp'].dt.month
data2['ph_months']=data2['mm'].between(4,9)
data2['ph']=((data2['tstp'].dt.hour.between(18,22)) & (data2['ph_months']))

In [103]:
ph_filter=(data2['ph']) & (data2['ph_months'])
ppw_ph=data2[ph_filter].groupby('month')['power'].max().apply(pd.Series)
ppw_non_ph=ppw_ph[0].nlargest(2).values.mean()

In [104]:
gr_data2=data2.groupby(['month','mm']).agg({'energy':sum,'power':max}).reset_index()

In [105]:
gr_data2=gr_data2.merge(ppw_ph.reset_index(),how='left').fillna(ppw_non_ph).rename(columns={0:'peak_power_ph','power':'max_power'})

In [106]:
def get_top_n_avg(lis,n):
    return np.mean(heapq.nlargest(n, lis))

In [107]:
gr_data2['roll']=[window.to_list() for window in gr_data2['max_power'].rolling(window=12)]
gr_data2['peak_power']=gr_data2['roll'].apply(lambda x: get_top_n_avg(x,2))
gr_data2.drop(columns=['roll'],inplace=True)

In [11]:
c1=chiTariffs.BT1(data,params_BT1)

NameError: name 'params_BT1' is not defined

In [10]:
c1.price()

,month,cfm,csp,tr,en,pw,bpw,pw_win,bpw_win,Total
0,2013-01-01,684.73,182.814976,4600.603649,28397.859400,5641.353474,6595.014274,0.0,0.0,46102.375773
1,2013-02-01,684.73,193.863976,4878.655652,30114.173666,5982.306477,6993.604777,0.0,0.0,48847.334548
2,2013-03-01,684.73,243.679472,6132.280255,37852.344116,7519.526382,8790.688982,0.0,0.0,61223.249207
3,2013-04-01,684.73,184.194704,4635.324998,28612.181707,5683.929496,6644.787697,0.0,0.0,46445.148603
4,2013-05-01,684.73,154.378152,3884.980897,23980.579442,4763.842346,5569.161446,0.0,0.0,39037.672283
5,2013-06-01,684.73,152.364948,3834.317904,23667.855141,4701.718356,5496.535506,0.0,0.0,38537.521855
6,2013-07-01,684.73,137.744708,3466.394383,21396.796553,4250.563074,4969.113224,0.0,0.0,34905.341942
7,2013-08-01,684.73,137.104628,3450.286543,21297.368693,4230.811314,4946.022464,0.0,0.0,34746.323642
8,2013-09-01,684.73,163.328096,4110.209409,25370.833440,5040.022114,5892.028914,0.0,0.0,41261.151973
9,2013-10-01,684.73,192.110868,4834.538060,29841.851757,5928.208591,6930.361741,0.0,0.0,48411.801017


In [6]:
data['date']=data['tstp'].dt.date.apply(lambda dt: dt.replace(day=1))

In [42]:
monthly_data=data.groupby(['date']).sum().reset_index()

In [73]:
monthly_data.rename(columns={'date':'month','energy(kWh/hh)':'energy'},inplace=True)